In [8]:
import cv2
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

# Load the CIFAR-10 dataset
with open('data_batch_2', 'rb') as f:
    data = pickle.load(f, encoding='bytes')

# Extract the images and labels from the dataset
images = data[b'data']
labels = data[b'labels']

# Convert the image data into images
images = [cv2.cvtColor(np.reshape(image, (3, 32, 32)).transpose(1, 2, 0), cv2.COLOR_BGR2RGB) for image in images]

# Reshape the images into 32x32x3 arrays and normalize them
images = np.array([cv2.resize(image, (32, 32)) for image in images])

# Extract color histograms as features
features = []
for image in images:
    if image.size == 0:
        continue
    histogram = cv2.calcHist([image], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    features.append(histogram.flatten())


# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2)


# Train the KNN classifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train)

# Evaluate the classifier on the test set
accuracy = classifier.score(X_test, y_test)
print('Accuracy: {:.2f}'.format(accuracy))


Accuracy: 0.28


In [11]:
import tensorflow as tf
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.model_selection import train_test_split

# Load the CIFAR-10 dataset
with open('data_batch_1', 'rb') as f:
    data = pickle.load(f, encoding='bytes')

# Extract the images and labels from the dataset
images = data[b'data']
labels = data[b'labels']

# Convert the image data into images
images = [cv2.cvtColor(np.reshape(image, (3, 32, 32)).transpose(1, 2, 0), cv2.COLOR_BGR2RGB) for image in images]

# Reshape the images into 32x32x3 arrays and normalize them
images = np.array([cv2.resize(image, (32, 32)) for image in images])

# Extract features using a CNN
model = tf.keras.applications.VGG16(include_top=False, weights='imagenet')
features = model.predict(images)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2)

# Build a DNN classifier
model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(1, 1, 512)))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

y_train = np.array(y_train)
y_test = np.array(y_test)

# Train the model
model.fit(X_train, y_train, epochs=5)

# Evaluate the model on the test set
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: {:.2f}'.format(accuracy))

313/313 [==============================] - 78s 249ms/step
Epoch 1/5
250/250 [==============================] - 3s 5ms/step - loss: 4.8409 - accuracy: 0.3156
Epoch 2/5
250/250 [==============================] - 1s 4ms/step - loss: 1.7602 - accuracy: 0.4136
Epoch 3/5
250/250 [==============================] - 1s 4ms/step - loss: 1.4422 - accuracy: 0.4959
Epoch 4/5
250/250 [==============================] - 1s 3ms/step - loss: 1.2513 - accuracy: 0.5506
Epoch 5/5
63/63 [==============================] - 1s 3ms/step - loss: 1.7377 - accuracy: 0.4490
Accuracy: 0.45


In [26]:
import tensorflow as tf
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.model_selection import train_test_split

# Load the CIFAR-10 dataset
with open('data_batch_1', 'rb') as f:
    data = pickle.load(f, encoding='bytes')

# Extract the images and labels from the dataset
images = data[b'data']
labels = data[b'labels']

# Convert the image data into images
images = [cv2.cvtColor(np.reshape(image, (3, 32, 32)).transpose(1, 2, 0), cv2.COLOR_BGR2RGB) for image in images]

# Reshape the images into 32x32x3 arrays and normalize them
images = np.array([cv2.resize(image, (32, 32)) for image in images])

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Preprocess the data
X_train = tf.keras.applications.vgg16.preprocess_input(X_train)
X_test = tf.keras.applications.vgg16.preprocess_input(X_test)

# Load the VGG-16 model
model = tf.keras.applications.VGG16(include_top=False, weights='imagenet')

# Freeze the layers of the model
for layer in model.layers:
    layer.trainable = False

# Extract features from the images using the frozen VGG-16 model
features_train = model.predict(X_train)
features_test = model.predict(X_test)

# Flatten the feature maps
features_train = features_train.reshape((len(X_train), 1, 1, 512))
features_test = features_test.reshape((len(X_test), 1, 1, 512))

# Reshape the input tensor to the target shape
input = tf.keras.layers.Input(shape=(1, 1, 512))
s = tf.reshape(input, (32, 32, 512))

# Build a CNN classifier
classifier = tf.keras.Sequential()
classifier.add(tf.keras.layers.Reshape((32, 32, 512), input_shape=(32, 32, 512)))
classifier.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu'))
classifier.add(tf.keras.layers.MaxPooling2D((2, 2)))
classifier.add(tf.keras.layers.Flatten())
classifier.add(tf.keras.layers.Dense(64, activation='relu'))
classifier.add(tf.keras.layers.Dense(10, activation='softmax'))

# Compile the model
classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
classifier.fit(np.array(features_train), np.array(y_train), epochs=5)

# Evaluate the model on the test set
_, accuracy = classifier.evaluate(np.array(features_test), np.array(y_test))
print('Accuracy: {:.2f}'.format(accuracy))


63/63 [==============================] - 12s 180ms/step
Epoch 1/5


ValueError: in user code:

    File "C:\Users\tgork\PycharmProjects\venv\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\tgork\PycharmProjects\venv\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\tgork\PycharmProjects\venv\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\tgork\PycharmProjects\venv\lib\site-packages\keras\engine\training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\tgork\PycharmProjects\venv\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\tgork\PycharmProjects\venv\lib\site-packages\keras\engine\input_spec.py", line 295, in assert_input